# Profit Maximization Project By Chaitanya Murarka
### Trying to Maximize the Operation Cost

In [1]:
import pandas as pd
from pulp import *
# num2words will tell our profits in words 
from num2words import num2words

#### Importing the datasets

In [2]:
demand = pd.read_csv("demand_forecasted.csv")
demand = demand.sort_values("Year")
periods = list(demand['Year']) #periods

In [3]:
distances = pd.read_csv('distances.csv')
distances['Costs']=distances['Distance']
distances = distances.drop(['Distance'],axis=1)
zones = list(distances['Region'].unique()) #zones
distances.head()

,Center,Region,Unnamed: 2,Costs
0,Center A,Downtown Toronto,NaN,25.77
1,Center A,East York,NaN,2.24
2,Center A,Etobicoke,NaN,25.72
3,Center A,Mississauga East,NaN,28.65
4,Center A,North Toronto,NaN,11.64


In [4]:
centers = pd.read_csv('centers.csv')
centers = centers.drop("Num MPs",axis=1)
depots = list(centers['Center']) #depots
centers = centers.set_index('Center')
#centers

In [5]:
production = pd.read_excel("production_cost.xlsx")
#production

In [6]:
selling = pd.read_excel("selling_price.xlsx")
selling = selling.set_index("zones")
#selling

#### Data Preprocessing

In [7]:
# Dictionaries about Depots
depotcost_open = {}
depotcost_opt = {}
depotcapacity = {}
for i in depots:
    depotcost_open[i]=float(centers.loc[i]["Opening Costs"])
    depotcost_opt[i] = float(centers.loc[i]["Operating Costs"])
    depotcapacity[i] = int(centers.loc[i]["Maximum capacity"])
    

In [8]:
# Dictionary about manufacting costs
depotmanufac = {}
for i in depots:
    for j in zones:
        depotmanufac[(i,j)] = float(production[(production["Center"]==i)&(production["Zones"]==j)]["Manufacturing Cost"])

In [9]:
# Dictionary about selling price
zonesell = {}
for j in zones:
    zonesell[j] = float(selling.loc[j]["selling price"])

In [10]:
# Dictionary about Depot to Zone Cost
depotzonecost = {}
for i in depots:
    for j in zones:
        depotzonecost[(i,j)] = float(distances[(distances["Center"] == i)&(distances["Region"] == j)]["Costs"])

In [11]:
# Dictionary about demand in each zone
zonedemand = {}
for t in periods:
    for j in zones:    
        zonedemand[(j,t)] = int(demand[(demand["Year"] == t)&(demand["Region"] == j)]['Demand'])


In [12]:
def selectyear(i):
    demand_copy = demand[demand['Year']==i]
    demand_copy = demand_copy.set_index('Region')
    zonedemand22 = {}
    for j in zones:    
        zonedemand22[j] = demand_copy.loc[j]["Demand"]
    return zonedemand22

## Optimizing

### Defining Optimization problem and decision variables

In [13]:
# Initialize Model and Decision Variable
z = LpProblem("Minimum-Transportation-Cost",LpMaximize)
y = LpVariable.dict('y',(depots),cat='Binary')
x = LpVariable.dict('x',(depots,zones),lowBound=0,cat='Continous')

### User Interface for Generalization

In [14]:
userinput1 = int(input("Please Enter the Year to check for the optimal solution :"))
zonedemand22 = selectyear(userinput1)
userinput2 = float(input("Plase Enter Maximum Production Capacity to Utilize"))

### Objective function

In [15]:
# Revenue Earned minus all possible expenses 

z +=  (lpSum(zonesell[j]*zonedemand22[j] for j in zones) -
      lpSum([ depotcost_opt[i] for i in ['Center A', 'Center B', 'Center C', 'Center D']] ) -
      lpSum([ (depotcost_open[i] + depotcost_opt[i]) * y[i] for i in ['Center E', 'Center F'] ]) -
      lpSum([ depotzonecost[(i,j)] * x[(i,j)] / 10000 for i in depots for j in zones ] )-
      lpSum(x[i,j] *depotmanufac[i,j] for i in depots for j in zones))


### Constriants

In [16]:

# Defining Constraints
from numpy import require


for j in zones:
    z += lpSum( [x[(i,j)] for i in depots] ) == zonedemand22[j]

# Constraint: depot capacities must be observed in all scenarios
for i in depots:        
    z += lpSum( [x[(i,j)] for j in zones] ) <= depotcapacity[i] * y[i]


# Capacity Utilization
#z += lpSum(zonedemand22[j] for j in zones) <= lpSum(depotcapacity[i] * y[i] for i in depots) * userinput2/100


In [17]:
#lpSum(depotcapacity[i] * y[i].varValue for i in depots)

### Solving and checking the status of solution

In [18]:
status = z.solve()
print("Solution is :",LpStatus[status])

Solution is : Optimal


### Printing the solution

In [19]:
print("The total cost is :",value(z.objective))
print("Words :",(num2words(value(z.objective),lang='en_IN')))


The total cost is : 8691651.532429
Words : eighty-six lakh, ninety-one thousand, six hundred and fifty-one point five three two four two nine


### Which Depot serves how much to which zone

In [20]:
for i in depots:
    if y[i].varValue >= 1.0:
        print(str(i) + " is open")                            
        for j in zones:
            if x[(i,j)].varValue > 0.0:
                print("\tServes zone " + str(j) , x[(i,j)].varValue ,"units")

Center A is open
	Serves zone East York 87165.0 units
	Serves zone North York 512835.0 units
Center B is open
	Serves zone East York 81592.0 units
	Serves zone Scarborough East 449380.0 units
Center C is open
	Serves zone Downtown Toronto 220428.0 units
	Serves zone Etobicoke 379572.0 units
Center D is open
	Serves zone East York 108421.0 units
	Serves zone Etobicoke 81836.0 units
	Serves zone Mississauga East 409743.0 units
Center E is open
	Serves zone Downtown Toronto 179851.0 units
	Serves zone North Toronto 420149.0 units


### Production Capacity Utilization

In [21]:
required = 0
for j in zonedemand22:
    required += zonedemand22[j]
production_capacity = 0
for i in depots:
        production_capacity += depotcapacity[i]*y[i].varValue
production_capacity_utilization = (required/production_capacity)*100
print("Production Capacity :",production_capacity)
print("Production Required :",required)
print("Capacity Utilized :",production_capacity_utilization,"%")


Production Capacity : 3000000.0
Production Required : 2930972
Capacity Utilized : 97.69906666666667 %
